# CNN with Embeddings on Post Content

In [ ]:
## Load libraries

In [1]:
# standard
import pandas as pd
import numpy as np
import random
import os

# tf and keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import models
from keras import layers
from keras.preprocessing.text import Tokenizer

from sklearn.preprocessing import LabelBinarizer

# plots
import seaborn as sns
import matplotlib.pyplot as plt

# for stop words
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jaredfeldman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jaredfeldman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data
training_data = pd.read_json("train.json")
training_data.head()

,giver_username_if_known,number_of_downvotes_of_request_at_retrieval,number_of_upvotes_of_request_at_retrieval,post_was_edited,request_id,request_number_of_comments_at_retrieval,request_text,request_text_edit_aware,request_title,requester_account_age_in_days_at_request,...,requester_received_pizza,requester_subreddits_at_request,requester_upvotes_minus_downvotes_at_request,requester_upvotes_minus_downvotes_at_retrieval,requester_upvotes_plus_downvotes_at_request,requester_upvotes_plus_downvotes_at_retrieval,requester_user_flair,requester_username,unix_timestamp_of_request,unix_timestamp_of_request_utc
0,N/A,0,1,0,t3_l25d7,0,Hi I am in need of food for my 4 children we a...,Hi I am in need of food for my 4 children we a...,Request Colorado Springs Help Us Please,0.000000,...,False,[],0,1,0,1,None,nickylvst,1317852607,1317849007
1,N/A,2,5,0,t3_rcb83,0,I spent the last money I had on gas today. Im ...,I spent the last money I had on gas today. Im ...,"[Request] California, No cash and I could use ...",501.111100,...,False,"[AskReddit, Eve, IAmA, MontereyBay, RandomKind...",34,4258,116,11168,None,fohacidal,1332652424,1332648824
2,N/A,0,3,0,t3_lpu5j,0,My girlfriend decided it would be a good idea ...,My girlfriend decided it would be a good idea ...,"[Request] Hungry couple in Dundee, Scotland wo...",0.000000,...,False,[],0,3,0,3,None,jacquibatman7,1319650094,1319646494
3,N/A,0,1,1,t3_mxvj3,4,"It's cold, I'n hungry, and to be completely ho...","It's cold, I'n hungry, and to be completely ho...","[Request] In Canada (Ontario), just got home f...",6.518438,...,False,"[AskReddit, DJs, IAmA, Random_Acts_Of_Pizza]",54,59,76,81,None,4on_the_floor,1322855434,1322855434
4,N/A,6,6,0,t3_1i6486,5,hey guys:\n I love this sub. I think it's grea...,hey guys:\n I love this sub. I think it's grea...,[Request] Old friend coming to visit. Would LO...,162.063252,...,False,"[GayBrosWeightLoss, RandomActsOfCookies, Rando...",1121,1225,1733,1887,None,Futuredogwalker,1373657691,1373654091


In [3]:
# isolate variables...
#X_training_data = np.array(training_data.drop('requester_received_pizza', axis = 1))
#print(X_training_data.shape)

# ...from labels
#Y_training_data = np.array(training_data['requester_received_pizza'])
#print(Y_training_data.shape)

In [4]:
training_data_text = training_data[['request_text_edit_aware', 'requester_received_pizza']]
training_data_text.dtypes

request_text_edit_aware     object
requester_received_pizza      bool
dtype: object

In [5]:
training_data_text

,request_text_edit_aware,requester_received_pizza
0,Hi I am in need of food for my 4 children we a...,False
1,I spent the last money I had on gas today. Im ...,False
2,My girlfriend decided it would be a good idea ...,False
3,"It's cold, I'n hungry, and to be completely ho...",False
4,hey guys:\n I love this sub. I think it's grea...,False
...,...,...
4035,Is anyone out there kind enough to help me out...,False
4036,If someone could hook me up with a $15 gift ca...,True
4037,"Have today off, soo I'll be stuck in the house...",False
4038,"I've never done anything like this before, but...",False


In [6]:
#<_BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [7]:
posts_pizza = training_data_text[training_data_text['requester_received_pizza'] == True]
posts_no_pizza = training_data_text[training_data_text['requester_received_pizza'] == False]

print(len(posts_pizza))
print(len(posts_no_pizza))
print(len(posts_pizza) + len(posts_no_pizza))


994
3046
4040


In [8]:
posts_pizza_list = list(posts_pizza['request_text_edit_aware'])
posts_no_pizza_list = list(posts_no_pizza['request_text_edit_aware'])

print(len(posts_pizza_list))
print(len(posts_no_pizza_list))
print(len(posts_pizza_list) + len(posts_no_pizza_list))

994
3046
4040


In [9]:
import io

# create files for each list
# uncomment below to run but files already created and moved

#for index, post in enumerate(posts_no_pizza_list):
    #print(index)
    #print(post)
    #with io.open("file_" + str(index) + ".txt", 'w', encoding = 'utf-8') as f:
        #f.write(post)

In [10]:
pwd

'/Users/jaredfeldman/Courses/MIDS207/free_pizza'

In [11]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'posts/posts_700', 
    batch_size=batch_size, 
    validation_split=0.1, 
    subset='training', 
    seed=seed)

Found 1401 files belonging to 2 classes.
Using 1261 files for training.


In [12]:
raw_train_ds

<_BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [13]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(3):
    print("Review", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

Review b"I have about 15.00 in Amazon payments credit and I don't feel like waiting for it to deposit into my account. I'm also craving some pizza...so, I was thinking I'd have some fun.\n\nWhoever orders me a pizza tonight and sends me their address will get a random gift chosen by me shipped to their house. It won't be a junk item, I will try to get creative and thoughtful with it. I won't tell you what it is so you'll just have to wait and see when it arrives.\n\nMy inspiration was a cross between RAOP and RAOA!"
Label 0
Review b'I would like to request pizza this evening. A friend of mine is coming over to drop off Dexter Season 1 and she is not bringing me any food. I would like to wwtch this show with my young teens and enjoy some Papa Johns or similar. If anyone can help, thanks in advance.'
Label 1
Review b"Pre-Law, if I make it through (wouldn't count on it if I were you but there's a chance) you get free law stuff. My ferrari lost its traction control and I lost my dads visa 

In [14]:
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])
#print("Label 2 corresponds to", raw_train_ds.class_names[2])

Label 0 corresponds to no_pizza_700
Label 1 corresponds to pizza_700


In [15]:
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'posts/posts_700', 
    batch_size=batch_size, 
    validation_split=0.1, 
    subset='validation', 
    seed=seed)

Found 1401 files belonging to 2 classes.
Using 140 files for validation.


In [16]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'posts/posts_700', 
    batch_size=batch_size)

Found 1401 files belonging to 2 classes.


In [17]:
import re
import string

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [18]:
max_features = 15000
sequence_length = 300

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [19]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

raw_train_ds

<_BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [20]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [21]:
# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b"Single Mom here. So, my oldest is almost 9. She came down with strep three days after school started back. Yay. =\\ Had a semi stressful weekend with trying to keep her and my youngest (almost 11 months) apart so the youngest wouldn't get infected too. However, the baby woke me up at 2am yesterday morning with a faucet for a nose and a slight cough. Called the doctor to see if that warranted a visit, and low and behold it does. She was also positive for strep. She's a really good baby though, and except for being off her bottle (I guess the sucking hurts?) a little bit, and hating to have her nose wiped, she's dealing amazingly well. \n\nSo to sum it up, I've got two sick kids and I'm behind on household chores (anyone got a laundry fairy they can spare??) and I'd really just like a night off from cooking dinner. I'm not desperate for food, and I totally understand if there are others who are and you'd rather help them. \n\nWe're in Metro Atlanta, and are happy with 

In [22]:
#print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
#print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

Vocabulary size: 7404


In [23]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [24]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [25]:
embedding_dim = 32

In [26]:
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          480032    
                                                                 
 dropout (Dropout)           (None, None, 32)          0         
                                                                 
 global_average_pooling1d (  (None, 32)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 480065 (1.83 MB)
Trainable params: 480065 (1.83 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [27]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 300), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [28]:
from tensorflow.keras import losses

model.compile(loss='binary_crossentropy', 
              optimizer=tf.keras.optimizers.legacy.Adam(learning_rate = 0.001),
              metrics=['accuracy'])


In [29]:
epochs = 10
history = model.fit(
    train_ds,
    batch_size = 32,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10
40/40 [==============================] - 2s 35ms/step - loss: 4.7767 - accuracy: 0.5028 - val_loss: 1.4630 - val_accuracy: 0.4786
Epoch 2/10
40/40 [==============================] - 1s 28ms/step - loss: 1.2927 - accuracy: 0.5028 - val_loss: 1.2029 - val_accuracy: 0.4786
Epoch 3/10
40/40 [==============================] - 1s 29ms/step - loss: 1.1124 - accuracy: 0.5028 - val_loss: 1.0708 - val_accuracy: 0.4786
Epoch 4/10
40/40 [==============================] - 1s 28ms/step - loss: 1.0003 - accuracy: 0.5028 - val_loss: 0.9658 - val_accuracy: 0.4786
Epoch 5/10
40/40 [==============================] - 1s 28ms/step - loss: 0.9129 - accuracy: 0.5028 - val_loss: 0.8836 - val_accuracy: 0.4786
Epoch 6/10
40/40 [==============================] - 1s 28ms/step - loss: 0.8459 - accuracy: 0.5028 - val_loss: 0.8214 - val_accuracy: 0.4786
Epoch 7/10
40/40 [==============================] - 1s 28ms/step - loss: 0.7961 - accuracy: 0.5028 - val_loss: 0.7761 - val_accuracy: 0.4786
Epoch 8/10
40

In [30]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

44/44 [==============================] - 0s 5ms/step - loss: 0.6891 - accuracy: 0.5054
Loss:  0.6891027688980103
Accuracy:  0.5053533315658569


In [37]:
# implement a learning rate schedule to reduce learning rate each epoch

def lr_schedule(epoch):
    initial_learning_rate = 0.001
    decay_steps = 5
    decay_rate = 0.5
    lr = initial_learning_rate * (decay_rate ** (epoch // decay_steps))
    return lr

# Create the LearningRateScheduler callback
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

In [38]:
def build_model():
  tf.keras.backend.clear_session()
  tf.random.set_seed(0)

  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(
      input_dim = max_features,  # size of feature vocabulary
      output_dim = 4,  # embedding dimension
      input_length = sequence_length  # number of inputs
      ))

  model.add(tf.keras.layers.Conv1D(filters=16,
                                   strides = 3,
                                   padding = 'same',
                                   kernel_size=12, # length of word
                                   activation='relu'))

  # Average over the sequence dimension, so each review is represented by 
  # 1 vector of size embedding_dimension
  model.add(tf.keras.layers.GlobalAveragePooling1D()) 

  # Alternatively, we could concatenate the embedding representations of 
  # all tokens in the movie review
  #model.add(tf.keras.layers.Flatten())

  model.add(tf.keras.layers.Dense(
      units=16,        
      activation='relu'))

  model.add(tf.keras.layers.Dense(
      units=1,        
      activation='sigmoid'))

  model.compile(loss='binary_crossentropy', 
                optimizer=tf.keras.optimizers.legacy.Adam(),#learning_rate = 0.001), comment out to use learning rate schedule
                metrics=['accuracy'])
  
  return model

In [39]:
model = build_model()

# Display the model layers.
display(model.layers)
display(model.summary())

# Retrieve the embeddings layer, which itself is wrapped in a list.
embeddings = model.layers[-1].get_weights()[0]


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 4)            60000     
                                                                 
 conv1d (Conv1D)             (None, 100, 16)           784       
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 61073 (238.57 KB)
Trainable params: 61073 (238.57 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

None

In [40]:
history = model.fit(
    train_ds,
    batch_size = 16,
    validation_data=val_ds,
    epochs=15,
    callbacks=[lr_scheduler])


Epoch 1/15
40/40 [==============================] - 2s 39ms/step - loss: 0.6939 - accuracy: 0.4980 - val_loss: 0.6930 - val_accuracy: 0.5643 - lr: 0.0010
Epoch 2/15
40/40 [==============================] - 1s 25ms/step - loss: 0.6927 - accuracy: 0.5353 - val_loss: 0.6929 - val_accuracy: 0.4857 - lr: 0.0010
Epoch 3/15
40/40 [==============================] - 1s 24ms/step - loss: 0.6920 - accuracy: 0.5599 - val_loss: 0.6926 - val_accuracy: 0.5357 - lr: 0.0010
Epoch 4/15
40/40 [==============================] - 1s 25ms/step - loss: 0.6898 - accuracy: 0.5765 - val_loss: 0.6915 - val_accuracy: 0.5286 - lr: 0.0010
Epoch 5/15
40/40 [==============================] - 1s 24ms/step - loss: 0.6825 - accuracy: 0.6098 - val_loss: 0.6898 - val_accuracy: 0.5429 - lr: 0.0010
Epoch 6/15
40/40 [==============================] - 1s 24ms/step - loss: 0.6689 - accuracy: 0.6384 - val_loss: 0.6890 - val_accuracy: 0.5429 - lr: 5.0000e-04
Epoch 7/15
40/40 [==============================] - 1s 24ms/step - loss:

In [35]:
test_data = pd.read_json("test.json")
test_data.head(1)

,giver_username_if_known,request_id,request_text_edit_aware,request_title,requester_account_age_in_days_at_request,requester_days_since_first_post_on_raop_at_request,requester_number_of_comments_at_request,requester_number_of_comments_in_raop_at_request,requester_number_of_posts_at_request,requester_number_of_posts_on_raop_at_request,requester_number_of_subreddits_at_request,requester_subreddits_at_request,requester_upvotes_minus_downvotes_at_request,requester_upvotes_plus_downvotes_at_request,requester_username,unix_timestamp_of_request,unix_timestamp_of_request_utc
0,N/A,t3_i8iy4,Hey all! It's about 95 degrees here and our ki...,[request] pregger gf 95 degree house and no fo...,42.083866,0.0,57,0,10,0,16,"[AskReddit, COents, Denver, DenverBroncos, Lib...",364,840,j_like,1308963419,1308959819


In [36]:
def output_csv(predictions):
  test_data['request_text_edit_aware'] = predictions.reshape(-1).round().astype(int) # turns 0.6 into 1, 0.4 into 0, etc. since we just want 1s and 0s
  display(df_test.head(10))
  filename = 'titanic_predictions.csv'
  df_test.to_csv(
      filename,
      columns=['PassengerId', 'Survived'],
      index=False)
  print('Saved as:', filename)